In [0]:
# !pip install pytelegrambotapi

In [0]:
# !pip3 install torch torchvision
# !pip3 install Pillow==6.2.2

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import sys
sys.path.append('/content/drive/My Drive/Colab Notebooks/DLS-project')

In [0]:
# это рядом лежащий модуль
import style_transfer as st

In [0]:
token = "===secret==="

In [0]:
import telebot
from io import BytesIO

In [0]:
from telebot import types

options_markup = types.ReplyKeyboardMarkup(row_width=2, resize_keyboard=True, one_time_keyboard=True)
options_markup_btn1 = types.KeyboardButton('Обычный')
options_markup_btn2 = types.KeyboardButton('GAN')
options_markup.add(options_markup_btn1, options_markup_btn2)

start_markup = types.ReplyKeyboardMarkup(row_width=1, resize_keyboard=True, one_time_keyboard=True)
start_markup_btn1 = types.KeyboardButton('Начать')
start_markup.add(start_markup_btn1)

hideBoard = types.ReplyKeyboardRemove()

In [0]:
file_path_tpl = "https://api.telegram.org/file/bot{token}/{file_path}"
chats_online = []
__chat_pictures = {}

In [0]:
bot = telebot.TeleBot(token)

In [0]:
def start(message):
    if message.chat.id not in chats_online:
        msg = bot.send_message(message.chat.id,
                               'Привет! Я бот, созданный для демонстрации выпускного проекта '
                               'курса DLS-2019-2020, автор - Михаил Сибирёв.\n '
                               'Для начала выберите режим работы: обычный style transfer или с применением GAN',
                               reply_markup=options_markup)
        chats_online.append(message.chat.id)
        bot.register_next_step_handler(msg, style_transfer)

In [0]:
def style_transfer(message):
    text = message.text.lower()
    if text == 'обычный':
        msg = bot.send_message(message.chat.id,
                               'Вы выбрали обычный вариант style transfer. Теперь пришлите первую картинку (контент).')
        bot.register_next_step_handler(msg, get_first_picture)
    elif text == 'gan':
        msg = bot.send_message(message.chat.id,
                               'GAN сейчас не работает, извините. :)', reply_markup=options_markup)
        bot.register_next_step_handler(msg, style_transfer)
    else:
        bot.send_message(message.chat.id, 'Неизвестная команда. Начните сначала.', reply_markup=options_markup)

In [0]:
def get_first_picture(message):
    if message.photo:
        file_id = message.photo[-1].file_id
        file_path = file_path_tpl.format(
            token=token,
            file_path=bot.get_file(file_id).file_path
        )
        __chat_pictures[message.chat.id] = file_path
        msg = bot.send_message(message.chat.id, 'Теперь вторую картинку (стиль).', reply_markup=hideBoard)
        bot.register_next_step_handler(msg, get_second_picture)
    else:
        msg = bot.send_message(message.chat.id, 'Нужна именно картинка', reply_markup=hideBoard)
        bot.register_next_step_handler(msg, get_first_picture)

In [0]:
def get_second_picture(message):
    if message.photo:
        file_path_1 = __chat_pictures[message.chat.id]

        file_id = message.photo[-1].file_id
        file_path_2 = file_path_tpl.format(
            token=token,
            file_path=bot.get_file(file_id).file_path
        )
        
        bot.send_message(message.chat.id, 'Придётся подождать примерно 3-5 минут...', reply_markup=hideBoard)
        image = st.StyleTransfer().run(file_path_1, file_path_2)

        bio = BytesIO()
        bio.name = 'tmp_image'
        image.save(bio, 'JPEG')
        bio.seek(0)
        msg = bot.send_photo(message.chat.id, bio, caption='Результат', reply_markup=hideBoard)
        try:
            chats_online.pop(message.chat.id)
        except:
            pass
    else:
        msg = bot.send_message(message.chat.id, 'Нужна именно картинка', reply_markup=hideBoard)
        bot.register_next_step_handler(msg, get_second_picture)  

In [0]:
@bot.message_handler(commands=['start'])
def start_handler(message):
    start(message)

In [0]:
@bot.message_handler(commands=['reset'])
def reset_handler(message):
    msg = bot.send_message(message.chat.id, 'Сброс.', reply_markup=hideBoard)
    try:
        chats_online.pop(message.chat.id)
    except:
        pass
    bot.register_next_step_handler(msg, start)

In [0]:
bot.polling()